In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
import pickle
nltk.download('stopwords')
pkl_filename = "theModel.pkl" # name for saving and loading model
data = pd.read_csv('tweet.csv')
X, y= data['Tweet'], data['class_label']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\2020\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#text preprocessing
documents = []
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [32]:
#Removeing stopword
stw = stopwords.words('english')
addi = ("&amp","arc",'https','co','t','amp','s')
stw.append(addi)
#Do Count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1200, min_df=9, max_df=0.8, stop_words=stw)
X = vectorizer.fit_transform(documents).toarray()
y = data['class_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
data.sample(5)

,Unnamed: 0,Id,User,Tweet,class_label
1265,1265,5.783936e+08,I AM TAKING CHARGE™,Empowering Life Transformation for YOU: https:...,NEGATIVE
1087,1087,1.380000e+18,DogeOfTheNorthwoods,@Brandyski2121 #BidensAmerica Do they need pr...,NEGATIVE
375,375,3.166948e+09,Fernando Ortíz,"¡Together And United, VS This Pandemic Of The ...",NEGATIVE
618,618,1.210000e+18,Christopher Bartlett,Research has been used to #shape how #dentistr...,NEGATIVE
1383,1383,1.545869e+09,Prasad A. Bedekar,Be your own Protector....\nAn Initiative by Sh...,NEGATIVE


In [28]:
# XGBoost model
#model = XGBClassifier(eta=0.5,gamma=0.14,max_depth=7, random_state=42)
model = XGBClassifier(eta=0.5,gamma=0.35,max_depth=7, random_state=42)
model.fit(X_train, y_train)
# make predictions for test data
predictt = model.predict(X_test)
print(classification_report(y_test ,predictt))
print(confusion_matrix(y_test, predictt))
accuracy = accuracy_score(y_test, predictt)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\Users\2020\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[22:45:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

    NEGATIVE       0.73      0.79      0.76       135
    POSITIVE       0.64      0.56      0.60        90

    accuracy                           0.70       225
   macro avg       0

In [29]:
#Multinomial
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=0.3)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(classification_report(y_test ,pred ))
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred) * 100.0)

              precision    recall  f1-score   support

    NEGATIVE       0.72      0.82      0.77       135
    POSITIVE       0.66      0.52      0.58        90

    accuracy                           0.70       225
   macro avg       0.69      0.67      0.68       225
weighted avg       0.70      0.70      0.69       225

Confusion Matrix: 
 [[111  24]
 [ 43  47]]

Accuracy:  70.22222222222221


In [30]:
#random forest
from sklearn.ensemble import RandomForestClassifier
#n_estimators=400, random_state=42,max_depth=70
#model = RandomForestClassifier(n_estimators=400, random_state=42,max_depth=40,min_samples_split=5)
model = RandomForestClassifier(n_estimators=400, random_state=42,max_depth=40,min_samples_split=7)
model.fit(X_train, y_train) 
y_pred = model.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print('Accuracy: ', accuracy_score(y_test,y_pred))

[[121  14]
 [ 47  43]]
              precision    recall  f1-score   support

    NEGATIVE       0.72      0.90      0.80       135
    POSITIVE       0.75      0.48      0.59        90

    accuracy                           0.73       225
   macro avg       0.74      0.69      0.69       225
weighted avg       0.73      0.73      0.71       225

Accuracy:  0.7288888888888889


In [6]:
#Save model
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [8]:
#Load model
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)

In [9]:
#Text processing for gettin input
data = pd.read_csv('tweet.csv')
X, y= data['Tweet'], data['class_label']
documents = []
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)
stw = stopwords.words('english')
addi = ("&amp","arc",'https','co','t','amp','s')
stw.append(addi)
vectorizer = CountVectorizer(max_features=1500, min_df=9, max_df=0.8, stop_words=stw)
X = vectorizer.fit_transform(documents).toarray()
def printPos():
    print("██████╗░░█████╗░░██████╗██╗████████╗██╗██╗░░░██╗███████╗\n"+
          "██╔══██╗██╔══██╗██╔════╝██║╚══██╔══╝██║██║░░░██║██╔════╝\n"+
          "██████╔╝██║░░██║╚█████╗░██║░░░██║░░░██║╚██╗░██╔╝█████╗░░\n"+
          "██╔═══╝░██║░░██║░╚═══██╗██║░░░██║░░░██║░╚████╔╝░██╔══╝░░\n"+
          "██║░░░░░╚█████╔╝██████╔╝██║░░░██║░░░██║░░╚██╔╝░░███████╗\n"+
          "╚═╝░░░░░░╚════╝░╚═════╝░╚═╝░░░╚═╝░░░╚═╝░░░╚═╝░░░╚══════╝")
def printNeg():
    print("███╗░░██╗███████╗░██████╗░░█████╗░████████╗██╗██╗░░░██╗███████╗\n"+
          "████╗░██║██╔════╝██╔════╝░██╔══██╗╚══██╔══╝██║██║░░░██║██╔════╝\n"+
          "██╔██╗██║█████╗░░██║░░██╗░███████║░░░██║░░░██║╚██╗░██╔╝█████╗░░\n"+
          "██║╚████║██╔══╝░░██║░░╚██╗██╔══██║░░░██║░░░██║░╚████╔╝░██╔══╝░░\n"+
          "██║░╚███║███████╗╚██████╔╝██║░░██║░░░██║░░░██║░░╚██╔╝░░███████╗\n"+
          "╚═╝░░╚══╝╚══════╝░╚═════╝░╚═╝░░╚═╝░░░╚═╝░░░╚═╝░░░╚═╝░░░╚══════╝")       


In [10]:
#Get input and predict the input
inputt = input("Enter your text")
print("Your Input :",inputt)
res = model.predict(vectorizer.transform([inputt]).toarray())
print("Your result\n")
if(res[0] == 'POSITIVE'):
    printPos()
else:
    printNeg()

Your Input : klkjok
Your result

███╗░░██╗███████╗░██████╗░░█████╗░████████╗██╗██╗░░░██╗███████╗
████╗░██║██╔════╝██╔════╝░██╔══██╗╚══██╔══╝██║██║░░░██║██╔════╝
██╔██╗██║█████╗░░██║░░██╗░███████║░░░██║░░░██║╚██╗░██╔╝█████╗░░
██║╚████║██╔══╝░░██║░░╚██╗██╔══██║░░░██║░░░██║░╚████╔╝░██╔══╝░░
██║░╚███║███████╗╚██████╔╝██║░░██║░░░██║░░░██║░░╚██╔╝░░███████╗
╚═╝░░╚══╝╚══════╝░╚═════╝░╚═╝░░╚═╝░░░╚═╝░░░╚═╝░░░╚═╝░░░╚══════╝
